S. Blanchet | IBM Data Science Professional Certificate

# Segmenting and Clustering Neighbourghoods in Toronto
The purpose of this notebook is to explore, segment, and cluster neighbourhoods in the city of Toronto to highlight the most popular places.

In [1]:
# import the needed packages

from bs4 import BeautifulSoup
import requests 

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from pandas.io.json import json_normalize 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


## I. Create the Toronto neighbourhoods dataset

### A. Scrape the Wikipedia page containing the postal codes, borough and neighbourhood names within the city of Toronto and transform it into a dataframe.

In [2]:
# Get the postal codes, borough and neighbourhood names from Wikipedia

req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
page = BeautifulSoup(req.text, 'lxml')

print(page)

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":876823784,"wgRevisionId":876823784,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","w

We can see the relevant data is contained in the first table.

In [3]:
# Select the relevant part in the page
page.table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

I now transform the data into a *pandas* dataframe. 

In [4]:
# Transform the data into a pandas dataframe
df = pd.read_html(str(page.table))[0]

# See the result
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### B. Clean and reshape the dataframe

The first row is not desired: it contains the column labels. I drop it and rename the columns.

In [5]:
# Drop the first row
df.drop(0, axis=0, inplace=True)

# Rename columns
df.columns = ['PostalCode', 'Borough', 'Neighbourhood'] 

df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [6]:
# Also verify the last 5 rows, just in case!
df.tail()

,PostalCode,Borough,Neighbourhood
285,M8Z,Etobicoke,Mimico NW
286,M8Z,Etobicoke,The Queensway West
287,M8Z,Etobicoke,Royal York South West
288,M8Z,Etobicoke,South of Bloor
289,M9Z,Not assigned,Not assigned


We only process cells with an assigned borough. Let's first see how many they are.

In [7]:
# Transform "Not assigned" value into "NaN" and count them
df.replace('Not assigned', np.nan, inplace=True)
notassigned = df.isnull() 

print('Not assigned in Borough:', notassigned['Borough'].value_counts())
print('Not assigned in Neighbourhood:', notassigned['Neighbourhood'].value_counts())

Not assigned in Borough: False    212
True      77
Name: Borough, dtype: int64
Not assigned in Neighbourhood: False    211
True      78
Name: Neighbourhood, dtype: int64


There are 77 cells that have not an assigned borough. We need to drop them.

There will be remaining 1 cell that has not an assigned neighbourhood.

In [8]:
# Drop cells where a borough is not assigned
df = df.dropna(subset= ['Borough'], how= 'any')

df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,NaN
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


The observation with a not assigned neighbourhood belongs to the borough "Queen's Park". I will assign it the same value.

In [9]:
# Replace "NaN" with "Queen's Park"
df['Neighbourhood'].fillna("Queen's Park", inplace=True)

In [10]:
df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


I now group data by postal codes so postal codes can stand for a key. 

In [11]:
# Aggregate the data according to postal codes  
df = df.groupby('PostalCode').agg({'Borough':'first', 'Neighbourhood': ', '.join,}).reset_index() 

df.head(3)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [12]:
# Restore the order of the columns
df = df[['PostalCode', 'Borough', 'Neighbourhood']]

df.head(3)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [13]:
# Count the rows and columns in the dataframe
df.shape

(103, 3)

### C. Enrich the dataframe with the geospatial coordinates

In [14]:
# Load the geospatial data file
!wget --quiet http://cocl.us/Geospatial_data -O Geospatial_data.csv
    
print('Geospatial_data file downloaded!')

Geospatial_data file downloaded!


In [15]:
# Transform it into a dataframe
coord = pd.read_csv('Geospatial_data.csv')

coord.head(3)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


We can see the postal codes column's label is 'Postal Code'. We need to change it to be the same as in the previous dataframe.

In [16]:
# Rename the column
coord.rename(columns = {'Postal Code': 'PostalCode'}, inplace=True)

coord.head(3)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711


Now I can join the two dataframes to create one that contains the whole information.

In [17]:
# Join the two dataframes with 'PostalCode' as a key
df = df.join(coord.set_index('PostalCode'), on='PostalCode')

df.head(3)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711


In [18]:
df.shape

(103, 5)

## II. Explore Toronto venues

### A. Visualize Toronto neighbourhoods

In [19]:
# Find the geospatial coordinates of Toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinates of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto City are 43.653963, -79.387207.


Now that we have the needed coordinates, we can create a map of Toronto with neighbourhoods superimposed on top.

In [20]:
# Create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to the map
for lat, lng, Borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<img src="https://raw.githubusercontent.com/StfBlanchet/Capstone/master/Toronto_neighbourhoods.png">

### B. Get the top venues in Toronto with FourSquare

FourSquare credentials and version are defined in the next cell (not shared).

In [21]:
CLIENT_ID = 'secret'  
CLIENT_SECRET = 'secret'  
VERSION = '20180604'

I create the GET request URL to get the venues in Toronto within a radius of 500 meters.<br/>

In [22]:
# Request
search_query = ''
radius = 500  
limit = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=secret&client_secret=secret&ll=43.653963,-79.387207&v=20180604&query=&radius=500&limit=100'

Now I can send the GET request and examine the results.

In [23]:
results = requests.get(url).json()
results

As we know all the information is in the *items* key, we can use a get_category_type function that extracts the category of the venue.

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Then I can clean the json and structure it into a *pandas* dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
toronto_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
toronto_venues = toronto_venues.loc[:, filtered_columns]

# filter the category for each row
toronto_venues['venue.categories'] = toronto_venues.apply(get_category_type, axis=1)

# clean columns
toronto_venues.columns = [col.split(".")[-1] for col in toronto_venues.columns]

toronto_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))

<b>At the moment of the initial request, Foursquare returned 71 venues.</b>

Now we create a function to send a request relative to all the borough venues in Toronto.

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Below the code to run the above function on each borough and create a new dataframe called *toronto_venues*:

In [ ]:
toronto_venues = getNearbyVenues(names=df['Borough'], latitudes=df['Latitude'], longitudes=df['Longitude'])

<b>NB. I saved this dataframe in order to avoid sending exceeding requests to the FourSquare API with my personal credentials (free account).<br/>
This dataframe is used as it is in the rest of the study...</b>

In [ ]:
toronto_venues.to_csv('toronto_venues.csv')

In [26]:
file = 'toronto_venues.csv'
toront_venues = pd.read_csv(file)
toront_venues.head(3)

,Unnamed: 0,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,2,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [27]:
toronto_venues = toront_venues.drop(['Unnamed: 0'], axis=1)
toronto_venues.head(3)

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [28]:
# Check how many venues were returned for each borough
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,116,116,116,116,116,116
Downtown Toronto,1281,1281,1281,1281,1281,1281
East Toronto,121,121,121,121,121,121
East York,75,75,75,75,75,75
Etobicoke,74,74,74,74,74,74
Mississauga,11,11,11,11,11,11
North York,246,246,246,246,246,246
Queen's Park,43,43,43,43,43,43
Scarborough,87,87,87,87,87,87


In [29]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


## III. Analyze and cluster boroughs

### A. Enrich and reshape the dataframe

To prepare the dataframe for running K-means, we need to split values in the form of columns and assign them dummies values.

In [31]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add borough column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# Move borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studi

Let'see the new dataframe size.

In [32]:
toronto_onehot.shape

(2254, 280)

I now group rows by borough and by taking the mean of the frequency of occurrence of each category.

In [33]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studi

Just to confirm the new size:

In [34]:
toronto_grouped.shape

(11, 280)

I want to preview each borough along with the top 5 most common venues.

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2            Park  0.06
3     Pizza Place  0.04
4            Café  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.04
3       Bakery  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.07
2  Italian Restaurant  0.04
3      Ice Cream Shop  0.04
4             Brewery  0.03


----East York----
                 venue  freq
0          Coffee Shop  0.05
1                 Park  0.05
2  Sporting Goods Shop  0.04
3         Burger Joint  0.04
4        Grocery Store  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.09
1        Sandwich Place  0.07
2           Coffee Shop  0.05
3              Pharmacy  0.05
4  Fast Food Restaurant  0.04


----Mississauga----
                       venue  freq
0                      Hotel  0.18
1 

Now we define a function that return the most common venues for each borough.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

I use that function to create a new dataframe containing the ten most common venues by borough.

In [37]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park
3,East York,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store


### B. Cluster boroughs

I run k-means to cluster the neighbourhood into 11 clusters.<br/>
In a separate notebook, I have tested many value of *k* and came to the conclusion that k=11 provides the less impurity. 

In [38]:
# Set number of clusters
kclusters = 11

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 2,  9,  8,  7,  6,  0, 10,  4,  5,  1], dtype=int32)

I then create a new dataframe that includes the cluster as well as the top 5 venues for each borough.

In [39]:
# Add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster', kmeans.labels_)

toronto = df

# Merge dataframes to add latitude/longitude for each neighborhood
toronto = toronto.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant


Let's visualize the resulting clusters.

In [40]:
# Create map of Toronto using latitude and longitude values
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Cluster']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

<img src="https://raw.githubusercontent.com/StfBlanchet/Capstone/master/Toronto_venues_clusters.png">

Finally, we examine clusters to identify and assign them a meaningful category so to provide information to the visitor.

In [41]:
toronto.loc[toronto['Cluster'] == 0, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
86,Mississauga,0,Coffee Shop,Hotel,American Restaurant,Mediterranean Restaurant,Gym / Fitness Center


In [42]:
toronto.loc[toronto['Cluster'] == 1, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
76,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant
77,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant
78,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant
82,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant
83,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant
84,West Toronto,1,Bar,Coffee Shop,Café,Bakery,Italian Restaurant


In [43]:
toronto.loc[toronto['Cluster'] == 2, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
45,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
46,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
47,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
48,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
49,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
63,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
64,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place
65,Central Toronto,2,Coffee Shop,Sandwich Place,Park,Café,Pizza Place


In [44]:
toronto.loc[toronto['Cluster'] == 3, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
73,York,3,Park,Pharmacy,Bakery,Convenience Store,Pizza Place
74,York,3,Park,Pharmacy,Bakery,Convenience Store,Pizza Place
80,York,3,Park,Pharmacy,Bakery,Convenience Store,Pizza Place
81,York,3,Park,Pharmacy,Bakery,Convenience Store,Pizza Place
98,York,3,Park,Pharmacy,Bakery,Convenience Store,Pizza Place


In [45]:
toronto.loc[toronto['Cluster'] == 4, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
85,Queen's Park,4,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Diner,Gym


In [46]:
toronto.loc[toronto['Cluster'] == 5, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
1,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
2,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
3,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
4,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
5,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
6,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
7,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
8,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant
9,Scarborough,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant


In [47]:
toronto.loc[toronto['Cluster'] == 6, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
88,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
89,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
90,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
91,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
92,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
93,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
94,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
95,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
99,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store
100,Etobicoke,6,Pizza Place,Sandwich Place,Coffee Shop,Pharmacy,Liquor Store


In [48]:
toronto.loc[toronto['Cluster'] == 7, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
35,East York,7,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store
36,East York,7,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store
38,East York,7,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store
39,East York,7,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store
40,East York,7,Coffee Shop,Park,Burger Joint,Sporting Goods Shop,Grocery Store


In [49]:
toronto.loc[toronto['Cluster'] == 8, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,East Toronto,8,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park
41,East Toronto,8,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park
42,East Toronto,8,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park
43,East Toronto,8,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park
87,East Toronto,8,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Park


In [50]:
toronto.loc[toronto['Cluster'] == 9, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
50,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
51,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
52,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
53,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
54,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
55,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
56,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
57,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
58,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery
59,Downtown Toronto,9,Coffee Shop,Café,Restaurant,Hotel,Bakery


In [51]:
toronto.loc[toronto['Cluster'] == 10, toronto.columns[[1] + list(range(5, toronto.shape[1]))]]

,Borough,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
18,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
19,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
20,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
21,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
22,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
23,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
24,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
25,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place
26,North York,10,Coffee Shop,Clothing Store,Fast Food Restaurant,Restaurant,Pizza Place


As we can see, we have **4 "pure" categories**: 'Bar', 'Park', 'Pizza Place' and 'Greek Restaurant', each being characteristic of a given borough. We have a walk in a **park in York**, take a drink in a **bar in West Toronto**, and enjoy a **pizza in Etobicoke** or a **Greek restaurant in East Toronto**.<br/><br/>
The 6 remaining categories are in fact **a same big one, that is 'Coffee Shop'**. This is the first most common venue (75%) in the 7 other boroughs. This is a poor information for a visitor who certainly wants to diversify its activities in a city like Toronto. So I chose to also **highlight the second most common venue in order to discriminate between boroughs**. <br/><br/>
That is all the more relevant that:<br/>
<ul>
    <li>in most cases, the frequencies relative to the first and second most common venues are very close, some of them even being equal (except for Queen's Park where Coffee Shop is definitely the top choice!)</li>
    <li>it is remarkable that those "hybrid" categories are also "pure", in the sense that the second most common venue is each time a unique value (e.g. Cluster 10 : 1/ 100% 'Coffee Shop' ;  2/ 100% 'Clothing Store'). </li>
    </ul>

In [52]:
# Have a closer look at the first most common venues
toronto['1st Most Common Venue'].describe()

count             103
unique              5
top       Coffee Shop
freq               75
Name: 1st Most Common Venue, dtype: object

I create a new column combining the first and second common venues by borough and call it 'Venues'. I will use those values as labels in the clusters map.

In [53]:
toronto['Venues'] = toronto[['1st Most Common Venue', '2nd Most Common Venue']].apply(lambda x: ' & '.join(x.astype(str)), axis=1)
toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Venues
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant,Coffee Shop & Fast Food Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant,Coffee Shop & Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant,Coffee Shop & Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant,Coffee Shop & Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,5,Coffee Shop,Fast Food Restaurant,Bakery,Pizza Place,Chinese Restaurant,Coffee Shop & Fast Food Restaurant


Now I can generate a new map that displays meaningful information.

In [54]:
# Create map of Toronto using latitude and longitude values
map_venues = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to map
markers_colors = []
for lat, lng, poi, cluster, Venues  in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Cluster'], toronto['Venues']):
    label = folium.Popup(str(poi) + ': ' + str(Venues), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_venues)
    
map_venues

<img src="https://raw.githubusercontent.com/StfBlanchet/Capstone/master/Toronto_clusters_meaningful.png">

Thank you for the review!